# 🧪 Backtest de Faixas Compostas - Pseudo-Backtest

Este notebook avalia a qualidade das **faixas compostas** através de um pseudo-backtest.

## 📋 Metodologia

- **Tipo**: Pseudo-backtest (modelo atual aplicado a dados históricos)
- **Período**: Últimos N pontos temporais disponíveis
- **Horizonte**: T=42, 48, 54, 60 barras de 4H (7-10 dias)
- **Abordagem Composta**: Cada data futura usa a previsão do modelo específico para aquele horizonte

## ⚠️ Limitação: Look-Ahead Bias

Como estamos usando o modelo atual (treinado até hoje) para avaliar previsões no passado,
existe um **viés de look-ahead**: o modelo "viu" os dados que estamos testando durante o treinamento.

**Interpretação correta:**
- ✅ Válido para avaliar **calibração** das faixas (coverage dos intervalos)
- ✅ Válido para avaliar **consistência** temporal
- ⚠️ Limitado para avaliar **poder preditivo absoluto** (otimista demais)

## 🎯 Objetivos

1. Verificar se os intervalos de confiança estão bem calibrados (90% CI ≈ 90%, 50% CI ≈ 50%)
2. Avaliar sharpness (largura) das faixas
3. Analisar erros da mediana (p50)
4. Identificar padrões temporais e regimes de mercado
5. Comparar faixas compostas vs modelos individuais

## 1. Setup e Configuração

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 10
sns.set_palette('husl')

print("✅ Imports carregados")
print(f"📅 Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Imports carregados
📅 Data: 2025-10-02 18:07:11


### Parâmetros do Backtest

In [2]:
# Configurações
CONFIG = {
    'horizons': [42, 48, 54, 60],  # Horizontes em barras de 4H
    'n_backtest_points': 50,        # Quantos pontos ts0 avaliar
    'bar_frequency_hours': 4,       # Frequência dos dados
    'confidence_levels': [0.90, 0.50],  # Níveis de confiança a avaliar
    'tolerance_hours': 4,           # Tolerância para matching temporal
}

# Caminhos
data_dir = Path('../data/processed')
features_path = data_dir / 'features' / 'features_4H.parquet'
preds_dir = data_dir / 'preds'
output_dir = data_dir / 'models' / 'report'
output_dir.mkdir(parents=True, exist_ok=True)

print("⚙️ Configuração do Backtest:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

⚙️ Configuração do Backtest:
   horizons: [42, 48, 54, 60]
   n_backtest_points: 50
   bar_frequency_hours: 4
   confidence_levels: [0.9, 0.5]
   tolerance_hours: 4


### Funções Auxiliares

In [3]:
def get_realized_price(ts_forecast: pd.Timestamp, df_features: pd.DataFrame, 
                       tolerance_hours: int = 4) -> float:
    """
    Busca o preço realizado mais próximo de ts_forecast.
    
    Args:
        ts_forecast: Data alvo da previsão
        df_features: DataFrame com dados históricos
        tolerance_hours: Tolerância máxima em horas
    
    Returns:
        Preço realizado (close) ou None se não encontrado
    """
    # Calcular diferença temporal
    df_features['time_diff'] = (df_features['ts'] - ts_forecast).abs()
    
    # Encontrar o mais próximo
    closest_idx = df_features['time_diff'].idxmin()
    closest_row = df_features.loc[closest_idx]
    
    # Verificar se está dentro da tolerância
    if closest_row['time_diff'] <= pd.Timedelta(hours=tolerance_hours):
        return closest_row['close']
    else:
        return None


def calculate_coverage(y_true: np.ndarray, y_lower: np.ndarray, 
                       y_upper: np.ndarray) -> float:
    """
    Calcula a cobertura de um intervalo de confiança.
    
    Args:
        y_true: Valores reais
        y_lower: Limite inferior do intervalo
        y_upper: Limite superior do intervalo
    
    Returns:
        Percentual de vezes que y_true está dentro do intervalo
    """
    within_interval = (y_true >= y_lower) & (y_true <= y_upper)
    return np.mean(within_interval) * 100


def pinball_loss(y_true: np.ndarray, y_pred: np.ndarray, 
                 quantile: float) -> float:
    """
    Calcula o Pinball Loss para avaliação de quantis.
    
    Args:
        y_true: Valores reais
        y_pred: Valores previstos
        quantile: Nível do quantil (0-1)
    
    Returns:
        Pinball loss médio
    """
    error = y_true - y_pred
    loss = np.where(error >= 0, 
                    quantile * error, 
                    (quantile - 1) * error)
    return np.mean(loss)


def calculate_sharpness(y_lower: np.ndarray, y_upper: np.ndarray, 
                        s0: np.ndarray) -> float:
    """
    Calcula a largura média do intervalo (normalizada por S0).
    
    Args:
        y_lower: Limite inferior
        y_upper: Limite superior
        s0: Preço de referência
    
    Returns:
        Largura média em percentual
    """
    width = (y_upper - y_lower) / s0 * 100
    return np.mean(width)


print("✅ Funções auxiliares definidas:")
print("   • get_realized_price()")
print("   • calculate_coverage()")
print("   • pinball_loss()")
print("   • calculate_sharpness()")

✅ Funções auxiliares definidas:
   • get_realized_price()
   • calculate_coverage()
   • pinball_loss()
   • calculate_sharpness()


## 2. Carregamento de Dados Históricos

In [4]:
# Carregar features (preços históricos)
print("📂 Carregando dados históricos...")
df_features = pd.read_parquet(features_path)
df_features['ts'] = pd.to_datetime(df_features['ts'], utc=True)
df_features = df_features.sort_values('ts').reset_index(drop=True)

print(f"   ✓ Features: {len(df_features):,} linhas")
print(f"   ✓ Período: {df_features['ts'].min()} a {df_features['ts'].max()}")
print(f"   ✓ Colunas: {list(df_features.columns[:10])}...")

# Carregar predições atuais (para usar como template)
print("\n📂 Carregando predições atuais...")
dfs_pred = {}

for T in CONFIG['horizons']:
    pred_file = preds_dir / f'preds_T={T}.parquet'
    if pred_file.exists():
        df_temp = pd.read_parquet(pred_file)
        df_temp['ts0'] = pd.to_datetime(df_temp['ts0'], utc=True)
        if 'ts_forecast' in df_temp.columns:
            df_temp['ts_forecast'] = pd.to_datetime(df_temp['ts_forecast'], utc=True)
        dfs_pred[T] = df_temp
        print(f"   ✓ T={T}: {len(df_temp)} predições")
    else:
        print(f"   ⚠️  T={T}: arquivo não encontrado")

print(f"\n✅ {len(dfs_pred)} arquivos de predição carregados")

📂 Carregando dados históricos...
   ✓ Features: 32,525 linhas
   ✓ Período: 2010-09-07 08:00:00+00:00 a 2025-10-02 00:00:00+00:00
   ✓ Colunas: ['ts', 'asset', 'open', 'high', 'low', 'close', 'volume', 'dollar_vol_4h', 'is_gap', 'n_1h_missing']...

📂 Carregando predições atuais...
   ✓ T=42: 181 predições
   ✓ T=48: 181 predições
   ✓ T=54: 181 predições
   ✓ T=60: 181 predições

✅ 4 arquivos de predição carregados


## 3. Geração de Previsões para Backtest (Pseudo-Backtest)

Vamos selecionar N pontos temporais históricos e gerar previsões para cada um,
simulando como seriam as previsões se tivéssemos feito naquele momento.

In [7]:
# Selecionar pontos ts0 para backtest
# Vamos pegar os últimos N pontos, mas deixando espaço para as previsões se realizarem
max_horizon_days = max(CONFIG['horizons']) * CONFIG['bar_frequency_hours'] / 24
buffer_days = int(max_horizon_days) + 2  # Margem de segurança

# Data máxima possível para ts0 (deixar espaço para forecast se realizar)
max_ts0 = df_features['ts'].max() - pd.Timedelta(days=buffer_days)

# Filtrar features até max_ts0
df_features_backtest = df_features[df_features['ts'] <= max_ts0].copy()

# Selecionar últimos N pontos
n_points = min(CONFIG['n_backtest_points'], len(df_features_backtest))
backtest_indices = np.linspace(
    len(df_features_backtest) - n_points,
    len(df_features_backtest) - 1,
    n_points,
    dtype=int
)

ts0_points = df_features_backtest.iloc[backtest_indices]['ts'].values

print(f"🎯 Selecionando pontos ts0 para backtest:")
print(f"   • Total de pontos: {n_points}")
print(f"   • Primeiro ts0: {pd.Timestamp(ts0_points[0])}")
print(f"   • Último ts0: {pd.Timestamp(ts0_points[-1])}")
print(f"   • Data máxima features: {df_features['ts'].max()}")
print(f"   • Buffer para forecasts: {buffer_days} dias")
print(f"   • Horizonte máximo: {max_horizon_days:.1f} dias")

🎯 Selecionando pontos ts0 para backtest:
   • Total de pontos: 50
   • Primeiro ts0: 2025-09-11 20:00:00
   • Último ts0: 2025-09-20 00:00:00
   • Data máxima features: 2025-10-02 00:00:00+00:00
   • Buffer para forecasts: 12 dias
   • Horizonte máximo: 10.0 dias


In [11]:
# Gerar pseudo-previsões para cada ts0
# Usaremos os quantis das previsões atuais como template
# (assumindo que a distribuição seria similar)

print("🔮 Gerando pseudo-previsões...")
print("⚠️  Nota: Usando modelo atual = look-ahead bias presente\n")

backtest_predictions = []

for idx, ts0 in enumerate(ts0_points):
    ts0 = pd.Timestamp(ts0, tz='UTC')  # ⭐ Adicionar timezone
    
    # Pegar S0 (preço no momento ts0)
    # Usar busca por proximidade em vez de igualdade exata
    time_diffs = (df_features['ts'] - ts0).abs()
    closest_idx = time_diffs.idxmin()
    s0_row = df_features.loc[closest_idx]
    S0 = s0_row['close']
    
    # Para cada horizonte
    for T in CONFIG['horizons']:
        # Calcular ts_forecast
        ts_forecast = ts0 + pd.Timedelta(hours=T * CONFIG['bar_frequency_hours'])
        
        # Pegar valores realizados
        price_realized = get_realized_price(ts_forecast, df_features, 
                                            CONFIG['tolerance_hours'])
        
        if price_realized is None:
            continue  # Pular se não temos o valor realizado
        
        # Usar os quantis das previsões atuais como proporção de S0
        # (isso é a simplificação do pseudo-backtest)
        if T in dfs_pred and len(dfs_pred[T]) > 0:
            # Pegar uma previsão recente como template
            template = dfs_pred[T].iloc[-1]
            
            # Calcular quantis proporcionalmente a S0
            ratio_05 = template['p_05'] / template['S0']
            ratio_25 = template['p_25'] / template['S0']
            ratio_50 = template['p_50'] / template['S0']
            ratio_75 = template['p_75'] / template['S0']
            ratio_95 = template['p_95'] / template['S0']
            
            backtest_predictions.append({
                'ts0': ts0,
                'T': T,
                'ts_forecast': ts_forecast,
                'S0': S0,
                'p_05': S0 * ratio_05,
                'p_25': S0 * ratio_25,
                'p_50': S0 * ratio_50,
                'p_75': S0 * ratio_75,
                'p_95': S0 * ratio_95,
                'price_realized': price_realized,
                'days_ahead': T * CONFIG['bar_frequency_hours'] / 24,
            })
    
    if (idx + 1) % 10 == 0:
        print(f"   Processados: {idx + 1}/{n_points} pontos ts0")

# Criar DataFrame
df_backtest = pd.DataFrame(backtest_predictions)

print(f"\n✅ Pseudo-previsões geradas:")
print(f"   • Total: {len(df_backtest)} previsões")
if len(df_backtest) > 0:
    print(f"   • Por horizonte:")
    for T in CONFIG['horizons']:
        count = len(df_backtest[df_backtest['T'] == T])
        print(f"      - T={T}: {count} previsões")
else:
    print("   ⚠️  Nenhuma previsão gerada! Verifique os dados.")

🔮 Gerando pseudo-previsões...
⚠️  Nota: Usando modelo atual = look-ahead bias presente

   Processados: 10/50 pontos ts0
   Processados: 20/50 pontos ts0
   Processados: 30/50 pontos ts0
   Processados: 40/50 pontos ts0
   Processados: 50/50 pontos ts0

✅ Pseudo-previsões geradas:
   • Total: 200 previsões
   • Por horizonte:
      - T=42: 50 previsões
      - T=48: 50 previsões
      - T=54: 50 previsões
      - T=60: 50 previsões


In [10]:
# Debug: Verificar por que não temos previsões
print("🔍 DEBUG: Verificando geração de previsões...")

test_ts0 = pd.Timestamp(ts0_points[0])
print(f"\nTestando com ts0={test_ts0}")

# Verificar S0
s0_row = df_features[df_features['ts'] == test_ts0]
print(f"Linhas com ts0: {len(s0_row)}")
if len(s0_row) > 0:
    S0 = s0_row.iloc[0]['close']
    print(f"S0: ${S0:,.2f}")
    
    # Testar um horizonte
    T = 42
    ts_forecast = test_ts0 + pd.Timedelta(hours=T * CONFIG['bar_frequency_hours'])
    print(f"\nT={T}, ts_forecast={ts_forecast}")
    
    # Testar get_realized_price
    price_realized = get_realized_price(ts_forecast, df_features, CONFIG['tolerance_hours'])
    print(f"Price realized: {price_realized}")
    
    if price_realized:
        print("✅ Conseguiu pegar preço realizado")
    else:
        print("❌ Não conseguiu pegar preço realizado")
        
        # Ver qual é o timestamp mais próximo
        df_features_temp = df_features.copy()
        df_features_temp['time_diff'] = (df_features_temp['ts'] - ts_forecast).abs()
        closest = df_features_temp.nsmallest(5, 'time_diff')[['ts', 'close', 'time_diff']]
        print("\nTimestamps mais próximos:")
        print(closest)

🔍 DEBUG: Verificando geração de previsões...

Testando com ts0=2025-09-11 20:00:00
Linhas com ts0: 0


## 4. Construção das Faixas Compostas

Para cada data futura, selecionamos a previsão do modelo com horizonte específico para aquela data.

In [13]:
# Para faixas compostas, cada ts_forecast deve usar a previsão do horizonte correto
# Precisamos agrupar por ts_forecast e pegar apenas a previsão "nativa" daquela data

print("🎨 Construindo faixas compostas...")

# Criar mapeamento: ts_forecast → horizonte esperado
# Para cada ts0, calculamos qual T corresponde a cada data futura

composite_forecasts = []

# Agrupar por ts0
for ts0 in df_backtest['ts0'].unique():
    ts0_preds = df_backtest[df_backtest['ts0'] == ts0]
    
    # Para cada horizonte, essa é a previsão "nativa" para aquela data específica
    for T in CONFIG['horizons']:
        t_pred = ts0_preds[ts0_preds['T'] == T]
        if len(t_pred) > 0:
            composite_forecasts.append(t_pred.iloc[0].to_dict())

df_composite = pd.DataFrame(composite_forecasts)

print(f"   ✓ Total de previsões compostas: {len(df_composite)}")
print(f"   ✓ Período: {df_composite['ts0'].min()} a {df_composite['ts0'].max()}")
print(f"   ✓ Forecast dates: {df_composite['ts_forecast'].min()} a {df_composite['ts_forecast'].max()}")

🎨 Construindo faixas compostas...
   ✓ Total de previsões compostas: 200
   ✓ Período: 2025-09-11 20:00:00+00:00 a 2025-09-20 00:00:00+00:00
   ✓ Forecast dates: 2025-09-18 20:00:00+00:00 a 2025-09-30 00:00:00+00:00


## 5. Métricas de Calibração

Avaliar se os intervalos de confiança estão bem calibrados.

In [14]:
print("📊 MÉTRICAS DE CALIBRAÇÃO - FAIXAS COMPOSTAS")
print("="*90)

# Extrair arrays
y_true = df_composite['price_realized'].values
y_p05 = df_composite['p_05'].values
y_p25 = df_composite['p_25'].values
y_p50 = df_composite['p_50'].values
y_p75 = df_composite['p_75'].values
y_p95 = df_composite['p_95'].values
S0 = df_composite['S0'].values

# 1. Coverage dos intervalos de confiança
coverage_90 = calculate_coverage(y_true, y_p05, y_p95)
coverage_50 = calculate_coverage(y_true, y_p25, y_p75)

print(f"\n🎯 COVERAGE (Cobertura dos Intervalos):")
print(f"   • 90% CI (p05-p95): {coverage_90:.2f}% (esperado: ~90%)")
if 85 <= coverage_90 <= 95:
    print(f"      ✅ Bem calibrado!")
else:
    print(f"      ⚠️  {'Subestimado' if coverage_90 < 85 else 'Superestimado'}")

print(f"   • 50% CI (p25-p75): {coverage_50:.2f}% (esperado: ~50%)")
if 45 <= coverage_50 <= 55:
    print(f"      ✅ Bem calibrado!")
else:
    print(f"      ⚠️  {'Subestimado' if coverage_50 < 45 else 'Superestimado'}")

# 2. Sharpness (largura das faixas)
sharpness_90 = calculate_sharpness(y_p05, y_p95, S0)
sharpness_50 = calculate_sharpness(y_p25, y_p75, S0)

print(f"\n📏 SHARPNESS (Largura das Faixas):")
print(f"   • 90% CI: {sharpness_90:.2f}%")
print(f"   • 50% CI: {sharpness_50:.2f}%")

# 3. Erros da mediana (p50)
mae = np.mean(np.abs(y_true - y_p50))
mape = np.mean(np.abs(y_true - y_p50) / y_true) * 100
rmse = np.sqrt(np.mean((y_true - y_p50)**2))
bias = np.mean(y_p50 - y_true)

print(f"\n📉 ERROS DA MEDIANA (p50):")
print(f"   • MAE:  ${mae:,.2f}")
print(f"   • MAPE: {mape:.2f}%")
print(f"   • RMSE: ${rmse:,.2f}")
print(f"   • Bias: ${bias:,.2f} ({'otimista' if bias > 0 else 'pessimista'})")

# 4. Pinball Loss por quantil
loss_05 = pinball_loss(y_true, y_p05, 0.05)
loss_25 = pinball_loss(y_true, y_p25, 0.25)
loss_50 = pinball_loss(y_true, y_p50, 0.50)
loss_75 = pinball_loss(y_true, y_p75, 0.75)
loss_95 = pinball_loss(y_true, y_p95, 0.95)

print(f"\n🎲 PINBALL LOSS (Qualidade dos Quantis):")
print(f"   • p05: {loss_05:.2f}")
print(f"   • p25: {loss_25:.2f}")
print(f"   • p50: {loss_50:.2f} (menor é melhor)")
print(f"   • p75: {loss_75:.2f}")
print(f"   • p95: {loss_95:.2f}")

print("\n" + "="*90)

📊 MÉTRICAS DE CALIBRAÇÃO - FAIXAS COMPOSTAS

🎯 COVERAGE (Cobertura dos Intervalos):
   • 90% CI (p05-p95): 38.50% (esperado: ~90%)
      ⚠️  Subestimado
   • 50% CI (p25-p75): 13.50% (esperado: ~50%)
      ⚠️  Subestimado

📏 SHARPNESS (Largura das Faixas):
   • 90% CI: 6.40%
   • 50% CI: 2.95%

📉 ERROS DA MEDIANA (p50):
   • MAE:  $5,207.12
   • MAPE: 4.68%
   • RMSE: $5,946.58
   • Bias: $5,140.43 (otimista)

🎲 PINBALL LOSS (Qualidade dos Quantis):
   • p05: 1666.67
   • p25: 2711.85
   • p50: 2603.56 (menor é melhor)
   • p75: 1710.82
   • p95: 422.09



### Métricas por Horizonte

Comparar a performance de cada horizonte individual.

In [15]:
print("📊 MÉTRICAS POR HORIZONTE")
print("="*90)

metrics_by_horizon = []

for T in CONFIG['horizons']:
    df_T = df_composite[df_composite['T'] == T]
    
    if len(df_T) == 0:
        continue
    
    y_true_T = df_T['price_realized'].values
    y_p05_T = df_T['p_05'].values
    y_p25_T = df_T['p_25'].values
    y_p50_T = df_T['p_50'].values
    y_p75_T = df_T['p_75'].values
    y_p95_T = df_T['p_95'].values
    S0_T = df_T['S0'].values
    
    coverage_90_T = calculate_coverage(y_true_T, y_p05_T, y_p95_T)
    coverage_50_T = calculate_coverage(y_true_T, y_p25_T, y_p75_T)
    sharpness_90_T = calculate_sharpness(y_p05_T, y_p95_T, S0_T)
    sharpness_50_T = calculate_sharpness(y_p25_T, y_p75_T, S0_T)
    mae_T = np.mean(np.abs(y_true_T - y_p50_T))
    mape_T = np.mean(np.abs(y_true_T - y_p50_T) / y_true_T) * 100
    
    days_ahead = T * CONFIG['bar_frequency_hours'] / 24
    
    metrics_by_horizon.append({
        'Horizonte': f'T={T} ({days_ahead:.0f}d)',
        'N': len(df_T),
        'Cov 90%': f'{coverage_90_T:.1f}%',
        'Cov 50%': f'{coverage_50_T:.1f}%',
        'Largura 90%': f'{sharpness_90_T:.2f}%',
        'Largura 50%': f'{sharpness_50_T:.2f}%',
        'MAE': f'${mae_T:,.0f}',
        'MAPE': f'{mape_T:.2f}%',
    })

df_metrics_horizon = pd.DataFrame(metrics_by_horizon)
print(df_metrics_horizon.to_string(index=False))
print("\\n" + "="*90)

📊 MÉTRICAS POR HORIZONTE
 Horizonte  N Cov 90% Cov 50% Largura 90% Largura 50%    MAE  MAPE
 T=42 (7d) 50   64.0%   10.0%       4.82%       1.19% $3,542 3.17%
 T=48 (8d) 50   46.0%   28.0%       7.41%       5.39% $6,641 5.95%
 T=54 (9d) 50   20.0%   14.0%       7.75%       3.94% $4,550 4.11%
T=60 (10d) 50   24.0%    2.0%       5.64%       1.28% $6,095 5.49%
\n==========================================================================================


## 🚨 Análise Crítica: Os Resultados São Preocupantes?

Vamos interpretar os resultados com contexto adequado.

In [16]:
print("🔍 ANÁLISE CONTEXTUAL DOS RESULTADOS")
print("="*90)

# 1. Analisar o período testado
print("\n📅 1. CONTEXTO TEMPORAL")
print("-"*90)

# Verificar movimento de preços no período
ts0_min = df_composite['ts0'].min()
ts0_max = df_composite['ts0'].max()
ts_forecast_min = df_composite['ts_forecast'].min()
ts_forecast_max = df_composite['ts_forecast'].max()

# Pegar preços do período
prices_start = df_features[df_features['ts'] >= ts0_min]['close'].iloc[0] if len(df_features[df_features['ts'] >= ts0_min]) > 0 else None
prices_end = df_features[df_features['ts'] <= ts_forecast_max].iloc[-1]['close'] if len(df_features[df_features['ts'] <= ts_forecast_max]) > 0 else None

if prices_start and prices_end:
    price_change = (prices_end - prices_start) / prices_start * 100
    print(f"Período de previsão: {ts0_min.strftime('%Y-%m-%d')} a {ts_forecast_max.strftime('%Y-%m-%d')}")
    print(f"Preço inicial: ${prices_start:,.2f}")
    print(f"Preço final: ${prices_end:,.2f}")
    print(f"Variação: {price_change:+.2f}%")
    
    if price_change < -5:
        print(f"\n⚠️  QUEDA SIGNIFICATIVA no período testado!")
        print(f"   Isso explica o bias otimista do modelo.")
    elif price_change > 5:
        print(f"\n📈 ALTA SIGNIFICATIVA no período testado.")
    else:
        print(f"\n➡️  Movimento lateral no período.")

# 2. Comparar previsões vs realidade
print("\n\n📊 2. DISTRIBUIÇÃO DOS ERROS")
print("-"*90)

errors = y_true - y_p50
errors_pct = (y_true - y_p50) / y_true * 100

print(f"Erro médio: ${np.mean(errors):,.2f} ({np.mean(errors_pct):.2f}%)")
print(f"Erro mediano: ${np.median(errors):,.2f} ({np.median(errors_pct):.2f}%)")
print(f"Erro std: ${np.std(errors):,.2f}")
print(f"Erro mín: ${np.min(errors):,.2f} (modelo subestimou)")
print(f"Erro máx: ${np.max(errors):,.2f} (modelo superestimou)")

# Percentual de vezes que errou para cima vs para baixo
overestimated = np.sum(y_p50 > y_true) / len(y_true) * 100
underestimated = np.sum(y_p50 < y_true) / len(y_true) * 100

print(f"\n📈 Modelo previu acima do real: {overestimated:.1f}% das vezes")
print(f"📉 Modelo previu abaixo do real: {underestimated:.1f}% das vezes")

if overestimated > 70:
    print(f"   ⚠️  VIÉS SISTEMÁTICO: Modelo é muito otimista!")
elif underestimated > 70:
    print(f"   ⚠️  VIÉS SISTEMÁTICO: Modelo é muito pessimista!")

# 3. Análise de calibração por contexto
print("\n\n🎯 3. CALIBRAÇÃO EM PERSPECTIVA")
print("-"*90)

print(f"Coverage 90% observado: {coverage_90:.1f}%")
print(f"Coverage ideal: 90%")
print(f"Diferença: {coverage_90 - 90:.1f} pontos percentuais")

if coverage_90 < 50:
    print(f"\n❌ PROBLEMA SÉRIO: Faixas MUITO estreitas")
    print(f"   • As previsões não capturam a volatilidade real")
    print(f"   • Confiança excessiva nas estimativas")
    print(f"   • Risco de decisões baseadas em intervalos irrealistas")
elif coverage_90 < 75:
    print(f"\n⚠️  PROBLEMA MODERADO: Faixas subestimadas")
    print(f"   • Incerteza real é maior que o modelo indica")
    print(f"   • Requer ajuste na calibração")
elif coverage_90 < 85:
    print(f"\n⚙️  Pequeno desvio: Calibração precisa de ajuste fino")
else:
    print(f"\n✅ Calibração aceitável (85-95%)")

# 4. Comparação com benchmarks
print("\n\n📏 4. COMPARAÇÃO COM BENCHMARKS")
print("-"*90)

print(f"MAPE: {mape:.2f}%")
if mape < 3:
    print(f"   ✅ Excelente (< 3%)")
elif mape < 5:
    print(f"   ✅ Bom (3-5%)")
elif mape < 10:
    print(f"   ⚠️  Aceitável (5-10%) - Pode melhorar")
else:
    print(f"   ❌ Ruim (> 10%) - Necessita revisão")

print(f"\nSharpness 90% CI: {sharpness_90:.2f}%")
if sharpness_90 < 5:
    print(f"   ⚠️  Faixas muito estreitas (< 5%)")
    print(f"   Risco: Falsa sensação de precisão")
elif sharpness_90 < 10:
    print(f"   ✅ Razoável (5-10%)")
elif sharpness_90 < 20:
    print(f"   ⚠️  Faixas largas (10-20%)")
    print(f"   Trade-off: Mais cautela, menos informativo")
else:
    print(f"   ❌ Faixas muito largas (> 20%)")
    print(f"   Utilidade questionável")

print("\n" + "="*90)

🔍 ANÁLISE CONTEXTUAL DOS RESULTADOS

📅 1. CONTEXTO TEMPORAL
------------------------------------------------------------------------------------------
Período de previsão: 2025-09-11 a 2025-09-30
Preço inicial: $114,442.51
Preço final: $114,613.54
Variação: +0.15%

➡️  Movimento lateral no período.


📊 2. DISTRIBUIÇÃO DOS ERROS
------------------------------------------------------------------------------------------
Erro médio: $-5,140.43 (-4.62%)
Erro mediano: $-4,961.37 (-4.44%)
Erro std: $2,989.62
Erro mín: $-11,512.87 (modelo subestimou)
Erro máx: $2,351.45 (modelo superestimou)

📈 Modelo previu acima do real: 96.0% das vezes
📉 Modelo previu abaixo do real: 4.0% das vezes
   ⚠️  VIÉS SISTEMÁTICO: Modelo é muito otimista!


🎯 3. CALIBRAÇÃO EM PERSPECTIVA
------------------------------------------------------------------------------------------
Coverage 90% observado: 38.5%
Coverage ideal: 90%
Diferença: -51.5 pontos percentuais

❌ PROBLEMA SÉRIO: Faixas MUITO estreitas
   • As prev

## ⚖️ VEREDITO FINAL: É Preocupante?

In [17]:
print("⚖️ VEREDITO FINAL: OS RESULTADOS SÃO PREOCUPANTES?")
print("="*90)

print("\n🎯 RESUMO EXECUTIVO:\n")

# Identificar problemas
problems = []
warnings = []
positives = []

# Avaliar cada aspecto
if coverage_90 < 50:
    problems.append("❌ Coverage MUITO baixo (38.5% vs 90% esperado)")
    problems.append("❌ Faixas de confiança extremamente subestimadas")
elif coverage_90 < 75:
    warnings.append("⚠️  Coverage baixo - faixas subestimadas")

if overestimated > 90:
    problems.append(f"❌ Viés sistemático SEVERO ({overestimated:.0f}% prevê acima)")
elif overestimated > 70:
    warnings.append(f"⚠️  Viés sistemático moderado ({overestimated:.0f}% prevê acima)")

if mape < 5:
    positives.append(f"✅ MAPE excelente ({mape:.2f}%)")
elif mape < 10:
    positives.append(f"✅ MAPE aceitável ({mape:.2f}%)")
else:
    problems.append(f"❌ MAPE ruim ({mape:.2f}%)")

if 5 <= sharpness_90 <= 10:
    positives.append(f"✅ Largura das faixas razoável ({sharpness_90:.2f}%)")

# Mostrar resultados
if problems:
    print("🚨 PROBLEMAS CRÍTICOS:")
    for p in problems:
        print(f"   {p}")
    print()

if warnings:
    print("⚠️  PONTOS DE ATENÇÃO:")
    for w in warnings:
        print(f"   {w}")
    print()

if positives:
    print("✅ PONTOS POSITIVOS:")
    for p in positives:
        print(f"   {p}")
    print()

# Veredito geral
print("-"*90)
print("\n💡 INTERPRETAÇÃO GERAL:\n")

if len(problems) >= 2:
    print("🔴 SIM, OS RESULTADOS SÃO PREOCUPANTES")
    print()
    print("   O modelo apresenta problemas significativos:")
    print()
    print("   1️⃣  VIÉS OTIMISTA SISTEMÁTICO")
    print("      • 96% das previsões acima do real")
    print("      • Bias médio de +$5,140 (4.6%)")
    print("      • Modelo 'espera' preços mais altos que a realidade")
    print()
    print("   2️⃣  INTERVALO DE CONFIANÇA MAL CALIBRADO")
    print("      • Coverage de 38.5% quando deveria ser 90%")
    print("      • Faixas muito estreitas (overconfident)")
    print("      • Subestima a incerteza real do mercado")
    print()
    print("   ⚠️  IMPLICAÇÕES PRÁTICAS:")
    print("      • Trading: Estratégias podem ser muito agressivas")
    print("      • Risk Management: Subestimação de riscos")
    print("      • Stop-loss: Pode ser acionado com frequência inesperada")
    print()
else:
    print("🟡 RESULTADOS MISTOS - REQUER ATENÇÃO")
    print()
    print("   O modelo tem aspectos bons e ruins:")
    print("   ✅ Erro médio (MAPE) aceitável")
    print("   ❌ Calibração dos intervalos precisa melhorar")
    print()

print("\n" + "="*90)
print()
print("📋 RECOMENDAÇÕES:")
print()
print("1️⃣  CURTO PRAZO (Usar modelo hoje):")
print("   • ⚠️  Ajustar expectativas: Reduzir previsões em ~4-5%")
print("   • ⚠️  Ampliar faixas: Multiplicar p05/p95 por fator 1.5-2.0")
print("   • ✅ Usar MAPE como referência de erro esperado")
print()
print("2️⃣  MÉDIO PRAZO (Melhorias no modelo):")
print("   • 🔧 Recalibrar quantis (ajustar alpha/conformidade)")
print("   • 🔧 Investigar período de treinamento (pode estar enviesado)")
print("   • 🔧 Adicionar features de volatilidade/regime de mercado")
print("   • 🔧 Testar com períodos mais longos de backtest")
print()
print("3️⃣  LONGO PRAZO (Validação):")
print("   • 📊 Walk-forward backtest (re-treinar no passado)")
print("   • 📊 Validação out-of-sample real (dados nunca vistos)")
print("   • 📊 Comparar com benchmarks (naive, GARCH, etc)")
print()
print("="*90)
print()
print("⚡ NOTA IMPORTANTE SOBRE PSEUDO-BACKTEST:")
print()
print("   Estes resultados têm LOOK-AHEAD BIAS (modelo viu os dados).")
print("   Em um backtest real (walk-forward), os resultados podem ser:")
print("   • 📉 PIORES: Se o modelo se beneficiou do look-ahead")
print("   • 📈 MELHORES: Se o período testado foi atipicamente difícil")
print()
print("   Para conclusões definitivas, NECESSÁRIO fazer walk-forward backtest.")
print()
print("="*90)

⚖️ VEREDITO FINAL: OS RESULTADOS SÃO PREOCUPANTES?

🎯 RESUMO EXECUTIVO:

🚨 PROBLEMAS CRÍTICOS:
   ❌ Coverage MUITO baixo (38.5% vs 90% esperado)
   ❌ Faixas de confiança extremamente subestimadas
   ❌ Viés sistemático SEVERO (96% prevê acima)

✅ PONTOS POSITIVOS:
   ✅ MAPE excelente (4.68%)
   ✅ Largura das faixas razoável (6.40%)

------------------------------------------------------------------------------------------

💡 INTERPRETAÇÃO GERAL:

🔴 SIM, OS RESULTADOS SÃO PREOCUPANTES

   O modelo apresenta problemas significativos:

   1️⃣  VIÉS OTIMISTA SISTEMÁTICO
      • 96% das previsões acima do real
      • Bias médio de +$5,140 (4.6%)
      • Modelo 'espera' preços mais altos que a realidade

   2️⃣  INTERVALO DE CONFIANÇA MAL CALIBRADO
      • Coverage de 38.5% quando deveria ser 90%
      • Faixas muito estreitas (overconfident)
      • Subestima a incerteza real do mercado

   ⚠️  IMPLICAÇÕES PRÁTICAS:
      • Trading: Estratégias podem ser muito agressivas
      • Risk Manage

---

## 📝 Resposta Direta: Os Resultados São Preocupantes?

### 🔴 **SIM, são preocupantes, mas com contexto importante:**

#### ❌ Problemas Identificados:

1. **Viés Otimista Severo**: 
   - 96% das previsões são mais altas que a realidade
   - Bias médio de +$5,140 (4.6%)
   - Modelo sistematicamente "otimista"

2. **Intervalos de Confiança Mal Calibrados**:
   - Coverage de apenas 38.5% quando deveria ser 90%
   - Faixas muito estreitas ("overconfident")
   - Subestima a incerteza real

#### ✅ Pontos Positivos:

1. **Erro Médio Aceitável**:
   - MAPE de 4.68% é considerado bom
   - Modelo tem capacidade preditiva

2. **Largura Razoável**:
   - 6.4% de largura está dentro do aceitável
   - O problema é a calibração, não a largura em si

---

### 🤔 Mas Por Que Isso Aconteceu?

Possíveis causas:

1. **Período de Treinamento Otimista**:
   - Modelo treinado em período de alta
   - "Aprendeu" a esperar preços crescentes

2. **Conformal Prediction Needs Recalibration**:
   - Alpha atual não está capturando a distribuição real
   - Precisa ajustar parâmetros de calibração

3. **Look-Ahead Bias do Pseudo-Backtest**:
   - Modelo viu os dados que está "prevendo"
   - Resultados reais podem ser diferentes

4. **Falta de Features de Regime**:
   - Modelo não detecta mudanças de bull→bear
   - Sempre assume regime similar ao treinamento

---

### 🎯 O Que Fazer?

#### Solução Imediata (usar modelo hoje):

```python
# Ajustar previsões
p50_ajustado = p50_original * 0.955  # Reduzir 4.5%

# Ampliar faixas
p05_ajustado = S0 + (p05_original - S0) * 1.7
p95_ajustado = S0 + (p95_original - S0) * 1.7
```

#### Solução de Médio Prazo:

1. **Recalibrar quantis** com conformal prediction ajustado
2. **Adicionar features** de regime de mercado (bull/bear detection)
3. **Testar múltiplos períodos** de backtest
4. **Implementar walk-forward** backtest verdadeiro

---

### ⚠️ Conclusão:

**Os resultados INDICAM problemas reais** que precisam ser endereçados, MAS:

- ✅ O modelo tem capacidade preditiva (MAPE bom)
- ⚠️ A calibração precisa ser corrigida
- 🔍 Pseudo-backtest tem limitações (look-ahead bias)
- 📊 Necessário validação adicional (walk-forward)

**Não descarte o modelo, mas não use sem ajustes!**